https://github.com/taesungp/contrastive-unpaired-translation  
До начала обучения нужно подключить гугл диск, чтобы сохранять чекпоинты и картинки там

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Проверяем видеокарту и cuda

In [ ]:
!nvidia-smi

Mon Nov 29 09:14:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


# Установка зависиомстей

In [ ]:
!git clone https://github.com/taesungp/contrastive-unpaired-translation CUT

Cloning into 'CUT'...
remote: Enumerating objects: 267, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 267 (delta 44), reused 30 (delta 28), pack-reused 198
Receiving objects: 100% (267/267), 17.91 MiB | 16.24 MiB/s, done.
Resolving deltas: 100% (135/135), done.


In [ ]:
cd CUT

/content/CUT


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 676 kB 17.4 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-py3-none-any.whl size=655250 sha256=6f5fb2d9b01c6e69c2284916e3041e92cd125c073f6e654226ceff8c7cc9fa9f
  Stored in directory: /root/.cache/pip/wheels/2d/d1/9b/cde923274eac9cbb6ff0d8c7c72fe30a3da9095a38fd50bbf1
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=63e24805d7a9fe167300ae771fd2aaac04b579cc3b3b71c49227e31d67d40286
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5710 sha256=142e317c2616c22239e0376466ac96abfd58200ace1954a8c0abea00a44f0afa
  Stored in directory: /root/.cache/pip/wheels/ac/5c/3a/a80e1c65880945c71fd833408cd1e9a8cb7e2f8f37620bb75b
Successfully built visdom GPUtil torchfile


# Тест на котах

CUT: 200 эпох с постоянным learning rate (lr), 200 эпох с уменьшением lr   
FastCUT: 150 эпох с постоянным lr, 50 эпох с уменьшением lr  
Для продолжения обучения нужно добавить команду `--continue_train --epoch_count X --epoch Y` где X - номер эпохи, с которой продолжается обучение, а Y - номер эпохи, который соответствует нужному чекпоинту (если указано latest, то последний чекпоинт).  
Взято [отсюда](https://github.com/taesungp/contrastive-unpaired-translation/issues/29)

## Копируем датасет с grumpy cat, отправляем вывод в dev/null

In [ ]:
#!bash ./datasets/download_cut_dataset.sh grumpifycat 1>/dev/null

## Запускаем обучение CUT/FastCUT с сохранением чекпоинта на гугл диске

In [ ]:
#!python train.py --dataroot ./datasets/grumpifycat --name grumpycat1_CUT --CUT_mode CUT --checkpoints /content/drive/MyDrive/GAN_backup
# --continue_train --epoch_count X --epoch Y
# --continue_train --epoch_count 99 --epoch latest

# Обучаем CUT на перенос стиля из ИК в RGB
## Копируем датасет из гугл диска в колаб  
Папка trainA - видимый диапазон  
Папка trainB - ИК диапазон  
B->A (ИК->видимый)  

In [ ]:
!rm -r /content/CUT_dataset

rm: cannot remove '/content/CUT_dataset': No such file or directory


In [ ]:
!cp /content/drive/MyDrive/GAN_Dataset /content/CUT_dataset -r

In [ ]:
'''
import cv2
import os
import PIL

dir = '/content/CUT_dataset/trainA/'
for imagePath in os.listdir(dir):
  PIL.Image.open(dir+imagePath).convert('L').save(dir+imagePath)

  image = cv2.imread(dir+imagePath, cv2.IMREAD_GRAYSCALE)
  os.remove(dir+imagePath)
  imagePath = imagePath[:-4] + '.png'
  #cv2.imwrite(dir+imagePath, cv2.cvtColor(image, cv2.COLOR_RGB2GRAY))
  cv2.imwrite(dir+imagePath, image)


dir = '/content/CUT_dataset/trainB/'
for imagePath in os.listdir(dir):
  PIL.Image.open(dir+imagePath).convert('L').save(dir+imagePath)

  image = cv2.imread(dir+imagePath, cv2.IMREAD_GRAYSCALE)
  os.remove(dir+imagePath)
  imagePath = imagePath[:-4] + '.png'
  #cv2.imwrite(dir+imagePath, cv2.cvtColor(image, cv2.COLOR_RGB2GRAY))
  cv2.imwrite(dir+imagePath, image)

'''

## Запускаем обучение
--input_nc - сколько каналов у изображения на входе (по умолчанию 3)    
--output_nc - сколько каналов у изображения на выходе (по умолчанию 3)  
--display_id -1 - не показывать результаты в окне браузера (так как visdom не работает в колабе)  
Последняя эпоха 234  


[Взято отсюда](https://github.com/taesungp/contrastive-unpaired-translation/issues/46)  
После клонирования открываем файл ./CUT/data/unaligned_dataset.py, изменяем 67 строку на:  
transform = get_transform(modified_opt, grayscale=(self.opt.input_nc == 1))  
Чтобы можно было работать с серыми изображениями

In [ ]:
!python train.py --dataroot /content/CUT_dataset --name rgb2ir_CUT5 --CUT_mode CUT --checkpoints /content/drive/MyDrive/GAN_backup --input_nc 1 --output_nc 1 --display_id -1 --continue_train --epoch_count 234 --epoch latest

----------------- Options ---------------
                 CUT_mode: CUT                           
               batch_size: 1                             
                    beta1: 0.5                           
                    beta2: 0.999                         
          checkpoints_dir: /content/drive/MyDrive/GAN_backup	[default: ./checkpoints]
           continue_train: True                          	[default: False]
                crop_size: 256                           
                 dataroot: /content/CUT_dataset          	[default: placeholder]
             dataset_mode: unaligned                     
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: -1                            	[default: None]
            display_ncols: 4                             
             display_port: 8097                          
    